In [5]:
using ITensors
using ITensors.HDF5
using ProgressMeter
using JLD2
using Plots

In [6]:
function DMRG2(N; θ)
    sites = siteinds("S=1", N)

    os = OpSum()
    for j=1:N-1
        os += cos(θ)*0.5,"S+",j,"S-",j+1
        os += cos(θ)*0.5,"S-",j,"S+",j+1
        os += cos(θ),"Sz",j,"Sz",j+1

        os += sin(θ)*0.25,"S+",j,"S-",j+1,"S+",j,"S-",j+1
        os += sin(θ)*0.25,"S+",j,"S-",j+1,"S-",j,"S+",j+1
        os += sin(θ)*0.5,"S+",j,"S-",j+1,"Sz",j,"Sz",j+1

        os += sin(θ)*0.25,"S-",j,"S+",j+1,"S+",j,"S-",j+1
        os += sin(θ)*0.25,"S-",j,"S+",j+1,"S-",j,"S+",j+1
        os += sin(θ)*0.5,"S-",j,"S+",j+1,"Sz",j,"Sz",j+1

        os += sin(θ)*0.5,"Sz",j,"Sz",j+1,"S+",j,"S-",j+1
        os += sin(θ)*0.5,"Sz",j,"Sz",j+1,"S-",j,"S+",j+1
        os += sin(θ),"Sz",j,"Sz",j+1,"Sz",j,"Sz",j+1
    end
    MPH = MPO(os,sites)
    ψ = randomMPS(sites,2)
    
    sweeps = Sweeps(25)
    setmaxdim!(sweeps, 5,5,10,10,10,25,25,25,40)
    setcutoff!(sweeps, 1E-7)
    setnoise!(sweeps, 1E-7, 1E-8, 1E-9, 0.0)
    
    E0, ψ0 = dmrg(MPH, ψ, sweeps, outputlevel = 0)
    
    return ψ0
end

DMRG2 (generic function with 1 method)

In [7]:
ψ = DMRG2(100, θ = atan(1/3))
f = h5open("AKLT_ground.h5","w")
write(f,"psi",ψ)
close(f)

In [7]:
angles = [-π/2; -π/3; -π/4; -0.5:0.05:0.3; atan(1/3); 0.35:0.05:0.5]
ψ = Vector{MPS}(undef, length(angles))
i = 1
@showprogress for θ in angles
    ψ[i] = DMRG2(100, θ = θ)
    i += 1
end

In [8]:
ITensors.op(::OpName"expSz",::SiteType"S=1") = 
[-1 0 0
  0 1 0
  0 0 -1]

In [9]:
I = [1:5;]
J = [100:-1:96;]
str_corr = Matrix{Float64}(undef, length(angles), length(I))

for i in 1:length(I)
    os = OpSum()
    os += "Sz",I[i]
    for j in I[i]+1:J[i]-1
        os *= "expSz",j
    end
    os *= "Sz",J[i]
    for k in 1:length(ψ)
        str = MPO(os,siteinds(ψ[k]))
        str_corr[k,i] = inner(ψ[k]', str, ψ[k])
    end
end

In [10]:
save_object("AKLT_params.jld2",angles)
save_object("string_correlator.jld2",str_corr)